In [ ]:
import pyodbc
import pandas as pd
from fuzzywuzzy import fuzz
from tqdm import tqdm
import numpy as np

In [ ]:
server = "............."
database = "............."

cnxn = pyodbc.connect("DRIVER={SQL Server};SERVER="+server+";DATABASE="+database)
cursor = cnxn.cursor()

query = "select * from ADDRESS_GAZ where PREMISE_DESCRIPTION like 'Commercial%'"

address_df = pd.read_sql(query, cnxn)

In [ ]:
print(f"This dataframe has {len(address_df)} entries")

In [ ]:
address_strings = []

for i in range(len(address_df)):

    string = " ".join(thing for thing in address_df.iloc[i, 5:10])

    address_strings.append(string)

In [ ]:
file_path = "C:\\Users\\agozacan\\OneDrive - Humberside Fire and Rescue Service\\RBIP Project\\EPC Data"

epc_df = pd.read_csv(file_path+"\\combined_epc_data.csv")

In [ ]:
epc_df.replace(np.nan, "", inplace=True)

In [ ]:
epc_strings = []

for i in range(len(epc_df)):

    string = " ".join(thing for thing in epc_df.iloc[i, 2:5])

    epc_strings.append(string)

In [ ]:
matching_indices = []

for i in tqdm(range(len(epc_strings))):

    fuzz_ratios = []

    for j in range(len(address_strings)):

        if epc_strings[i][-8:] == address_strings[j][-8:]:

            fuzz_ratios.append(fuzz.token_set_ratio(epc_strings[i], address_strings[j]))

        else:

            fuzz_ratios.append(0)

    index = fuzz_ratios.index(max(fuzz_ratios))

    matching_indices.append(index)

In [ ]:
epc_df.insert(0, "UPRN", list(address_df.iloc[matching_indices, 1]))

In [ ]:
for i in range(len(epc_df)):

    if epc_df.UPRN[i] == 100052200498.0:

        epc_df.UPRN[i] = ""

In [ ]:
epc_df.to_csv("epc_UPRN.csv", index=False)